**Next Steps**

- Try with and without stopword removal

- Reduce incoming to FC layer
    - Decomplicate network

- verify cutout not happening in predictions, try more aggresive cutout

- Softmax to Sigmoid

- Wide / Deep / Max Pool

- class_weight = {0: 1.,1: 50., 2: 2.}

- optimal thresholding (Optimal thresholding for F1)

- Remove top 2% misclassified



**Aces**

- dropout and regularization
- OLR+SGD+momentum or LR Scheduling
- Architecture and Multiple Receptive Field
    - Go Wide
- HyperParams from Params Section
- Try with and without removing hypen `-`





**Features**
- Average Rating of Drug
- don't do stop words
- Use Drug Column through embedding layer as well to get Drug Vector.
- Run Fasttext/Glove on this corpus


In [1]:
import os
import gc
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";

# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  
 
# Do other imports now...
import keras

import tensorflow as tf
tf.test.is_gpu_available()

from keras import backend as K
K.tensorflow_backend._get_available_gpus()



Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session
import tensorflow

# Reset Keras Session
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
        del model
    except:
        pass

    print(gc.collect()) # if it's done something you should see a number being outputted

    # use the same config as you used to create the session
    config = tensorflow.ConfigProto(log_device_placement=True, allow_soft_placement=True,)
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tensorflow.Session(config=config))
    

In [3]:
reset_keras()

0


# Imports



In [189]:
from keras import backend as K
from keras.layers import Reshape, Multiply
import time
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import time

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report
import numpy as np
from nltk import word_tokenize
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout, SpatialDropout1D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D

from time import time
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols
from random import sample

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc
from sklearn.decomposition import PCA

import sys
import os
sys.path.append(os.getcwd())
from importlib import reload


import warnings
warnings.filterwarnings('ignore')

reload(model_utils)

import data_science_utils.vision.keras
reload(data_science_utils.vision.keras)
from data_science_utils.vision.keras import *

from data_science_utils.models import mean_absolute_percentage_error
from data_science_utils.models import median_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import seaborn as sns
from more_itertools import flatten
import dill
from collections import Counter
import operator
from gensim.models import FastText
import itertools
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.models.fasttext import load_facebook_model

import lib
reload(lib)
from lib import *

import params
reload(params)
from params import *

import data_gen
reload(data_gen)
from data_gen import *



from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.ensemble import RandomForestClassifier

from keras.regularizers import l2
from keras.regularizers import L1L2



<module 'data_science_utils.models' from '/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/data_science_utils/models/__init__.py'>

<module 'data_science_utils.vision.keras' from '/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/data_science_utils/vision/keras/__init__.py'>

<module 'lib' from '/home/ec2-user/SageMaker/lib.py'>

<module 'params' from '/home/ec2-user/SageMaker/params.py'>

<module 'data_gen' from '/home/ec2-user/SageMaker/data_gen.py'>

In [5]:
glove = api.load("glove-twitter-25")

In [297]:

fname = os.getcwd()+"/fasttext.model"
from gensim.models import FastText
fb_model = FastText.load(fname)

In [6]:
df = read_csv("train_padded.csv")
df_test = read_csv("test_padded.csv")


df.shape
df_test.shape

(5279, 22)

(2924, 20)

In [7]:
# How many words in test set are not in train set

train_words = set(list(more_itertools.flatten(df.full_txt.values)))
test_words = set(list(more_itertools.flatten(df_test.full_txt.values)))
all_words = set(list(more_itertools.flatten(df.full_txt.values))+list(more_itertools.flatten(df_test.full_txt.values)))
len(train_words)
len(test_words)
test_exlusive_words = test_words-train_words

print("Test Set Exclusive Distinct words = ",len(test_exlusive_words))

print("Total vocab size = ",len(all_words))


25548

18769

Test Set Exclusive Distinct words =  4267
Total vocab size =  29815


# PreProcess for M1 and M5

## Glove Matrix

In [8]:

le_train,le_transform, le = get_text_le(vocab_size=vocab_size, min_count=min_count)
_ = le_train(list(df.full_txt.values)+list(df_test.full_txt.values))

df["full_txt_encoded"] = le_transform(df.full_txt.values)
df_test["full_txt_encoded"] = le_transform(df_test.full_txt.values)

df["context_txt_encoded"] = le_transform(df.context_txt.values)
df_test["context_txt_encoded"] = le_transform(df_test.context_txt.values)


 19%|█▉        | 1020/5279 [00:00<00:00, 10195.17it/s]

Total Words before Min frequency filtering 29815
Total Words after Min frequency filtering 11859


100%|██████████| 2924/2924 [00:00<00:00, 7111.22it/s] 


In [31]:
embedding_matrix = np.random.rand(vocab_size, embedding_dims)*0.1

In [32]:
not_printed=True
for w in le["wd"].keys():
    try:
        embedding_matrix[le["wd"][w]] = glove.wv[w][:embedding_dims]
        if not_printed:
            not_printed=False
            print(w,le["wd"][w])
            print(embedding_matrix[le["wd"][w]])
    except:
        pass


patient 7
[-1.3836      0.71107    -1.83200002  0.65379     0.74203002 -0.1154
  0.47209999 -0.58218998  0.034721    0.17285    -0.51932001  0.26039001
 -2.91829991  0.67479998  0.38541001  0.23262     0.39111     0.49118999
 -0.043787   -1.05869997  0.71371001 -0.11127    -1.02769995  0.13327
 -0.60834998]


## One Cycle

In [200]:

from keras.callbacks import Callback
from keras import backend as K


# Code is ported from https://github.com/fastai/fastai
class OneCycleLR(Callback):
    def __init__(self,
                 epochs,
                 batch_size,
                 samples,
                 steps,
                 max_lr,
                 end_percentage=0.1,
                 scale=100,
                 maximum_momentum=0.95,
                 minimum_momentum=0.85,
                 verbose=True):
        """ This callback implements a cyclical learning rate policy (CLR).
        This is a special case of Cyclic Learning Rates, where we have only 1 cycle.
        After the completion of 1 cycle, the learning rate will decrease rapidly to
        100th its initial lowest value.
        # Arguments:
            max_lr: Float. Initial learning rate. This also sets the
                starting learning rate (which will be 10x smaller than
                this), and will increase to this value during the first cycle.
            end_percentage: Float. The percentage of all the epochs of training
                that will be dedicated to sharply decreasing the learning
                rate after the completion of 1 cycle. Must be between 0 and 1.
            scale_percentage: Float or None. If float, must be between 0 and 1.
                If None, it will compute the scale_percentage automatically
                based on the `end_percentage`.
            maximum_momentum: Optional. Sets the maximum momentum (initial)
                value, which gradually drops to its lowest value in half-cycle,
                then gradually increases again to stay constant at this max value.
                Can only be used with SGD Optimizer.
            minimum_momentum: Optional. Sets the minimum momentum at the end of
                the half-cycle. Can only be used with SGD Optimizer.
            verbose: Bool. Whether to print the current learning rate after every
                epoch.
        # Reference
            - [A disciplined approach to neural network hyper-parameters: Part 1 -- learning rate, batch size, weight_decay, and weight decay](https://arxiv.org/abs/1803.09820)
            - [Super-Convergence: Very Fast Training of Residual Networks Using Large Learning Rates](https://arxiv.org/abs/1708.07120)
        """
        super(OneCycleLR, self).__init__()

        if end_percentage < 0. or end_percentage > 1.:
            raise ValueError("`end_percentage` must be between 0 and 1")


        self.initial_lr = max_lr
        self.end_percentage = end_percentage
        self.scale = scale
        self.max_momentum = maximum_momentum
        self.min_momentum = minimum_momentum
        self.verbose = verbose

        if self.max_momentum is not None and self.min_momentum is not None:
            self._update_momentum = True
        else:
            self._update_momentum = False

        self.clr_iterations = 0.
        self.history = {}

        self.epochs = epochs
        self.batch_size = batch_size
        self.samples = samples
        self.steps = steps
        self.num_iterations = None
        self.mid_cycle_id = None

    def _reset(self):
        """
        Reset the callback.
        """
        self.clr_iterations = 0.
        self.history = {}

    def compute_lr(self):
        """
        Compute the learning rate based on which phase of the cycle it is in.
        - If in the first half of training, the learning rate gradually increases.
        - If in the second half of training, the learning rate gradually decreases.
        - If in the final `end_percentage` portion of training, the learning rate
            is quickly reduced to near 100th of the original min learning rate.
        # Returns:
            the new learning rate
        """
        if self.clr_iterations > 2 * self.mid_cycle_id:
            current_percentage = (self.clr_iterations - 2 * self.mid_cycle_id)
            current_percentage /= float((self.num_iterations - 2 * self.mid_cycle_id))
            new_lr = self.initial_lr * (1. + (current_percentage * (1. - 100.) / 100.)) / self.scale

        elif self.clr_iterations > self.mid_cycle_id:
            current_percentage = 1. - (
                    self.clr_iterations - self.mid_cycle_id) / self.mid_cycle_id
            new_lr = self.initial_lr * (1. + current_percentage * (self.scale - 1.)) / self.scale

        else:
            current_percentage = self.clr_iterations / self.mid_cycle_id
            new_lr = self.initial_lr * (1. + current_percentage * (self.scale - 1.)) / self.scale

        if self.clr_iterations == self.num_iterations:
            self.clr_iterations = 0

        return new_lr

    def compute_momentum(self):
        """
         Compute the momentum based on which phase of the cycle it is in.
        - If in the first half of training, the momentum gradually decreases.
        - If in the second half of training, the momentum gradually increases.
        - If in the final `end_percentage` portion of training, the momentum value
            is kept constant at the maximum initial value.
        # Returns:
            the new momentum value
        """
        if self.clr_iterations > 2 * self.mid_cycle_id:
            new_momentum = self.max_momentum

        elif self.clr_iterations > self.mid_cycle_id:
            current_percentage = 1. - ((self.clr_iterations - self.mid_cycle_id) / float(
                self.mid_cycle_id))
            new_momentum = self.max_momentum - current_percentage * (
                    self.max_momentum - self.min_momentum)

        else:
            current_percentage = self.clr_iterations / float(self.mid_cycle_id)
            new_momentum = self.max_momentum - current_percentage * (
                    self.max_momentum - self.min_momentum)

        return new_momentum

    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.steps is not None:
            self.num_iterations = self.epochs * self.steps
        else:
            if (self.samples % self.batch_size) == 0:
                remainder = 0
            else:
                remainder = 1
            self.num_iterations = (self.epochs + remainder) * self.samples // self.batch_size

        self.mid_cycle_id = int(self.num_iterations * ((1. - self.end_percentage)) / float(2))

        self._reset()
        K.set_value(self.model.optimizer.lr, self.compute_lr())

        if self._update_momentum:
            if not hasattr(self.model.optimizer, 'momentum'):
                raise ValueError("Momentum can be updated only on SGD optimizer !")

            new_momentum = self.compute_momentum()
            K.set_value(self.model.optimizer.momentum, new_momentum)

    def on_batch_end(self, epoch, logs=None):
        logs = logs or {}

        self.clr_iterations += 1
        new_lr = self.compute_lr()

        self.history.setdefault('lr', []).append(
            K.get_value(self.model.optimizer.lr))
        K.set_value(self.model.optimizer.lr, new_lr)

        if self._update_momentum:
            if not hasattr(self.model.optimizer, 'momentum'):
                raise ValueError("Momentum can be updated only on SGD optimizer !")

            new_momentum = self.compute_momentum()

            self.history.setdefault('momentum', []).append(
                K.get_value(self.model.optimizer.momentum))
            K.set_value(self.model.optimizer.momentum, new_momentum)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):
        if self.verbose:
            if self._update_momentum:
                print(" - lr: %0.5f - momentum: %0.2f " %
                      (self.history['lr'][-1], self.history['momentum'][-1]))

            else:
                print(" - lr: %0.5f " % (self.history['lr'][-1]))



# Modelling Functions

In [11]:
def dense_block(x,layer_width,dropout=0.1):
    x1 = conv_layer(x,n_kernels=16*layer_width,kernel_size=5,padding='same')
    x2 = conv_layer(x1,n_kernels=16*layer_width,kernel_size=3,padding='same')
    x = concatenate([x1,x2])
    x3 = conv_layer(x,n_kernels=8*layer_width,kernel_size=3,padding='same')
    
    x = concatenate([x1,x2,x3])
    x = transition_layer(x, n_kernels=16*layer_width, dropout=dropout)
    return x
    

In [12]:
def conv_pipe(x,layer_width,dropout=0):
    
    x1 = dense_block(x,layer_width)
    xg0 = GlobalAveragePooling1D()(x1)
    x2 = dense_block(x1,layer_width)
    xg1 = GlobalAveragePooling1D()(x2)
    
    # x3 = MaxPooling1D()(x2)
    x4 = dense_block(x2,layer_width)
    xg2 = GlobalAveragePooling1D()(x4)
    return concatenate([xg0,xg1, xg2])



In [144]:
def conv_pipe(x,layer_width,dropout=0):
    
    x1 = dense_block(x,int(1.25*layer_width))
    xg0 = GlobalAveragePooling1D()(x1)
    x2 = MaxPooling1D()(x1)
    x2 = dense_block(x2,layer_width)
    xg1 = GlobalAveragePooling1D()(x2)
    
    # x3 = MaxPooling1D()(x2)
    return concatenate([xg0,xg1])

## Resampler

In [13]:
def resample_input_for_training(x):
    xs1 = x[x['sentiment']==0]
    xs2 = x[x['sentiment']==1]
    x = pd.concat([x,xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1),xs1.sample(frac=1)])
    x = pd.concat([x,xs2.sample(frac=1),xs2.sample(frac=1),xs2.sample(frac=1),xs2.sample(frac=1),xs2.sample(frac=1)])
    x = x.sample(frac=1)
    
    return x


## Trainer

In [137]:
def train_generator_model_with_validation(model,train,val=None,extract_values=None,weights=None,
                                          epochs=None, batch_size=None,
                                          policy=None,lr=0.01,max_lr_olr=0.005, 
                                          lr_shed_fn=lambda e,lr:0.95*lr,
                                         df_test = None):
    # policy olr+sgd | sgd+lrs | adam
    train_copy = train.copy()
    train = resample_input_for_training(train)
    train_iterator = MakeIter(train,extract_values,batch_size)
    print("Before Resampling, Size = ",train_copy.shape,"\n",train_copy.sentiment.value_counts())
    print("After Resampling = ",train.shape,"\n",train.sentiment.value_counts())
    checkpoint = ModelCheckpoint("model.hdf5", monitor='acc', verbose=0, save_best_only=True, mode='max')
    assert epochs is not None
    assert policy is not None
    assert batch_size is not None
    assert weights is not None
    
    if policy == "olr":
        lr_manager = OneCycleLR(samples=train.shape[0], epochs=epochs, batch_size=batch_size,
                            steps=len(train_iterator), max_lr=max_lr_olr,
                            end_percentage=0.2,
                            maximum_momentum=None, minimum_momentum=None)
        callbacks=[lr_manager]
    else:
        callbacks = [LearningRateScheduler(lr_decay)] if lr_shed_fn is not None else []
    
    if policy in ["olr","sgd"]:
        optimizer = SGD(lr=lr, decay=0, momentum=0.9, nesterov=True)
    else:
        optimizer = Adam(lr=lr)
    
    model.compile(loss=["categorical_crossentropy"],
                  optimizer=optimizer)
    # print(model.summary())
    print("Total Model Params = ",model.count_params())
#     class_weight = {0: 6.2,1: 4.5, 2: 1.}
#     class_weight = {0: 2,1: 1.5, 2: 1.}
    if val is None:
        metrics = DataGenMetrics(train = MakeIter(train_copy,extract_values,batch_size),
                             val = None, weights = weights, df_test = df_test, extract_values = extract_values)
        callbacks.append(metrics)
        train_history = model.fit_generator(train_iterator,
                        steps_per_epoch=len(train_iterator), 
                        epochs=epochs, verbose=1,
                        callbacks=callbacks)
        return metrics
    else:
        validation_iterator = MakeIter(val,extract_values,batch_size)
        metrics = DataGenMetrics(train = MakeIter(train_copy,extract_values,batch_size),
                             val = MakeIter(val,extract_values,batch_size), weights = weights)
        callbacks.append(metrics)
        train_history = model.fit_generator(train_iterator,
                        steps_per_epoch=len(train_iterator), 
                        validation_data = validation_iterator, 
                        validation_steps = len(validation_iterator),
                        epochs=epochs, verbose=2,
                        callbacks=callbacks)
    

    
    

In [130]:
def looped_random_validation(df,model_fn, training_fn,extract_values,weights=[1,1,1],
                             epochs=None, batch_size=None,
                            policy=None,lr=0.01,max_lr_olr=0.005, lr_shed_fn=lambda e,lr:0.95*lr):
    
    
    assert policy is not None
    assert epochs is not None
    assert batch_size is not None
    model = model_fn()

    np.random.seed(8*91 + 3*19+np.random.randint(5617))
    df_train, df_val = train_test_split(df,test_size=0.3)
    training_fn(model,df_train,df_val,extract_values,weights,
               epochs=epochs, batch_size=batch_size, policy=policy,lr=lr,max_lr_olr=max_lr_olr, lr_shed_fn=lr_shed_fn)

    y_train_preds = model.predict(extract_test_wrapper(extract_values(df_train)))
    
    
    
    y_train_preds_out = np.argmax(y_train_preds, axis=1)
    _ = show_results(df_train['sentiment'], y_train_preds_out)

    
    y_val_preds = model.predict(extract_test_wrapper(extract_values(df_val)))
    y_val_preds_out = np.argmax(y_val_preds, axis=1)
    acc,f1_test = show_results(df_val['sentiment'], y_val_preds_out)
    
    find_best_caliberation(df_train['sentiment'],df_val['sentiment'],y_train_preds,y_val_preds)
    
    print(classification_report(df_val['sentiment'], y_val_preds_out))
    
    
    
        
    
        

## Extractor

In [16]:
def extract_test_wrapper(X_Y):
    return X_Y[0]
    
def extract_values(x):
    x1 = np.stack(x["full_txt_encoded"].values, axis=0)
    x2 = np.stack(x["full_txt_mask"].values, axis=0)
    x3 = np.stack(x["full_txt_mask_gaussian"].values, axis=0)
    x4 = np.stack(x["context_txt_encoded"].values, axis=0)
    x5 = x[["occurences_count","review_length","review_length_by_occurences_count","first_occurence","last_occurence"]].values
    
    x6 = np.stack(x['word_sentiments'].values, axis=0)
    x7 = np.stack(x['word_count'].values, axis=0)
    
    x8 = np.stack(x['word_sentiments_ctx'].values, axis=0)
    x9 = np.stack(x['word_count_ctx'].values, axis=0)
    
    
    
    
    # print(x1.shape,x2.shape,x3.shape,x4.shape,x5.shape)
    train_phase = False
    if 'ohe_labels' in x.columns:
        train_phase = True
        y = np.stack(x['ohe_labels'].values,axis=0)
    
    
    if train_phase:
        x1,x2,x3,x6,x7 = batch_cutout( x1,x2,x3,x6,x7, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)
        x4,x8,x9 = batch_cutout(x4,x8,x9, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)
    else:
        print("Test Predictions - No cutout")
    # print(x1.shape,x2.shape,x3.shape,x4.shape,x5.shape)
    if train_phase:
        return [x1,x2,x3,x4,x5, x6, x7, x8, x9],y
    else:
        return [x1,x2,x3,x4,x5, x6, x7, x8, x9]


# Model

In [190]:
def build_model():
    full_txt_input = Input(shape=(full_txt_maxlen,), dtype='int32')
    full_txt_mask = Input(shape=(full_txt_maxlen,), dtype='float32')
    full_txt_gaussian = Input(shape=(full_txt_maxlen,), dtype='float32')
    context_txt_input = Input(shape=(full_txt_maxlen,), dtype='int32')
    
    numeric_inputs = Input(shape=(5,),name="numeric_input", dtype='float32')
    
    full_txt_sentiments = Input(shape=(full_txt_maxlen,), dtype='float32')
    full_txt_counts = Input(shape=(full_txt_maxlen,), dtype='float32')
    
    context_txt_sentiments = Input(shape=(full_txt_maxlen,), dtype='float32')
    context_txt_counts = Input(shape=(full_txt_maxlen,), dtype='float32')
    
    
    fc_layer_width = conv_embedded_params["fc_layer_width"]
    full_text_conv_layer_width = conv_embedded_params["full_text_conv_layer_width"]
    context_conv_layer_width = conv_embedded_params["context_conv_layer_width"]
    fc_layer_depth = conv_embedded_params["fc_layer_depth"]
    lstm_context_units = conv_embedded_params["lstm_context_units"]
    lstm_full_text_units = conv_embedded_params["lstm_full_text_units"]
    
    
    embedding_layer = Embedding(max_features,
                    embedding_dims,
                    weights=[embedding_matrix],
                    input_length=full_txt_maxlen,
                    name="txt_embedding",)
    
    
    
    full_txt_embedding_out = embedding_layer(full_txt_input)
    context_txt_embedding_out = embedding_layer(context_txt_input) # 25 channels
    
    # https://github.com/keras-team/keras/issues/5474
    # print("Before Channel select, ",K.int_shape(context_txt_input),K.int_shape(context_txt_embedding_out))
    context_txt_input_select = Lambda(lambda x : x[:,full_txt_maxlen-context_txt_maxlen:full_txt_maxlen])(context_txt_input)
    context_txt_embedding_out = Lambda(lambda x : x[:,full_txt_maxlen-context_txt_maxlen:full_txt_maxlen,:])(context_txt_embedding_out)
    context_txt_sentiments_out = Lambda(lambda x : x[:,full_txt_maxlen-context_txt_maxlen:full_txt_maxlen])(context_txt_sentiments)
    context_txt_counts_out = Lambda(lambda x : x[:,full_txt_maxlen-context_txt_maxlen:full_txt_maxlen])(context_txt_counts)
    
    # print("After Channel select, ",K.int_shape(context_txt_input_select),K.int_shape(context_txt_embedding_out))
    
    # print("Pre Masking, ",K.int_shape(full_txt_embedding_out),K.int_shape(context_txt_embedding_out))
    
    # https://github.com/keras-team/keras/issues/9311
    
    def mask_zeros(ip):
        txt = ip[0]
        txt_out = ip[1]
        mask = K.cast(K.not_equal(txt, 0), 'float32')
        expanded_mask = K.tile(K.expand_dims(mask, axis=2), n=[1, 1, embedding_dims])
        out = Multiply()([expanded_mask,  txt_out])
        return out
    
    full_txt_embedding_out = Lambda(mask_zeros)([full_txt_input, full_txt_embedding_out])
    context_txt_embedding_out = Lambda(mask_zeros)([context_txt_input_select, context_txt_embedding_out])
    
    # print("Post Masking, ",K.int_shape(full_txt_embedding_out),K.int_shape(context_txt_embedding_out))
    
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    # print(K.int_shape(full_txt_gaussian),K.int_shape(full_txt_embedding_out))
    
    full_txt_sentiments_out = Reshape((full_txt_maxlen,1))(full_txt_sentiments) 
    full_txt_counts_out = Reshape((full_txt_maxlen,1))(full_txt_counts) 
    context_txt_sentiments_out = Reshape((context_txt_maxlen,1))(context_txt_sentiments_out) 
    context_txt_counts_out = Reshape((context_txt_maxlen,1))(context_txt_counts_out) 
    
    
    full_txt_embedding_out = concatenate([full_txt_sentiments_out,full_txt_embedding_out,full_txt_counts_out])
    context_txt_embedding_out = concatenate([context_txt_sentiments_out,context_txt_embedding_out,context_txt_counts_out])
    
    
    gaussian_mask_out = Lambda(lambda x: x[0] * K.expand_dims(x[1], axis=-1))([full_txt_embedding_out, full_txt_gaussian])
    full_txt_gaussian_out = Reshape((full_txt_maxlen,1))(full_txt_gaussian) 
    full_txt_mask_out = Reshape((full_txt_maxlen,1))(full_txt_mask) 
    
    fc_input0 = GlobalAveragePooling1D()(context_txt_embedding_out)
    fc_input0 = fc_layer(fc_input0,int(fc_layer_width/4),)
    
    
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,int(fc_layer_width/4),)
    
    
    full_txt_embedding_out = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    full_txt_embedding_out = Dropout(0.3)(full_txt_embedding_out)
    context_txt_embedding_out = Dropout(0.3)(context_txt_embedding_out)
    
    
    
    fc_input2 = conv_pipe(context_txt_embedding_out, context_conv_layer_width)
    fc_input3 = conv_pipe(full_txt_embedding_out, full_text_conv_layer_width)
    
#     fc_input4 = Bidirectional(GRU(lstm_context_units,dropout=0.1, return_sequences=True, implementation=2, unroll=True))(context_txt_embedding_out)
#     fc_input5 = Bidirectional(GRU(lstm_full_text_units, dropout=0.1, return_sequences=True, implementation=2, unroll=True))(full_txt_embedding_out)
#     fc_input4 = Bidirectional(GRU(int(lstm_context_units/2), implementation=2, unroll=True))(fc_input4)
#     fc_input5 = Bidirectional(GRU(int(lstm_full_text_units/2), implementation=2, unroll=True))(fc_input5)
    
    
    numerics = fc_layer(numeric_inputs,12,)
    numerics = fc_layer(numerics,8,)
    numerics = Dropout(0.2)(numerics)
    
    fc_input = concatenate([numerics,fc_input0,fc_input1,fc_input2,fc_input3])
    fc_input = Dropout(0.1)(fc_input)
    print("FC Input dims = ", K.int_shape(fc_input))
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
        fc_input = Dropout(0.1)(fc_input)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    # vs sigmoid
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs,
                         full_txt_sentiments,full_txt_counts,context_txt_sentiments,context_txt_counts], 
                  outputs=[out])
    
    return model

In [192]:
reset_keras()

119


In [ ]:
gc.collect()

conv_embedded_params
looped_random_validation(df,build_model,train_generator_model_with_validation,extract_values,
                        **conv_embedded_params["training_policy"])

0

{'full_text_conv_layer_width': 8,
 'context_conv_layer_width': 6,
 'lstm_full_text_units': 16,
 'lstm_context_units': 8,
 'fc_layer_width': 96,
 'fc_layer_depth': 2,
 'training_policy': {'epochs': 6,
  'batch_size': 64,
  'policy': 'adam',
  'lr': 0.003,
  'max_lr_olr': 0.1,
  'lr_shed_fn': <function params.lr_decay(epoch, prev_lr)>}}

FC Input dims =  (None, 552)
Before Resampling, Size =  (3695, 22) 
 2    2654
1     598
0     443
Name: sentiment, dtype: int64
After Resampling =  (9786, 22) 
 1    3588
0    3544
2    2654
Name: sentiment, dtype: int64
Total Model Params =  1092551
Epoch 1/6
Epoch = 1, Prev LR = 0.0030, New LR = 0.0002
 - 46s - loss: 1.2611 - acc: 0.3433 - val_loss: 1.2365 - val_acc: 0.2364
 - 31s - loss: 1.1014 - acc: 0.4218 - val_loss: 1.3610 - val_acc: 0.2399
train-test f1 correlation = 
           f1_train   f1_test
f1_train  1.000000  0.991046
f1_test   0.991046  1.000000


,p1_by_p0,p2_by_p0,acc_train,f1_train,acc_test,f1_test
2550,1.000000,1.000000,0.249374,0.247189,0.234965,0.233941
318,0.392745,2.082666,0.663053,0.432506,0.680420,0.425400
624,0.616994,2.082666,0.640534,0.452152,0.638462,0.424245
573,0.572250,2.082666,0.650820,0.456143,0.648252,0.423194
420,0.456563,2.082666,0.658882,0.437096,0.672028,0.422949


Epoch 3/6
Epoch = 3, Prev LR = 0.0006, New LR = 0.0030
 - 30s - loss: 0.9263 - acc: 0.5542 - val_loss: 1.4945 - val_acc: 0.3038
train-test f1 correlation = 
           f1_train   f1_test
f1_train  1.000000  0.979825
f1_test   0.979825  1.000000


...

Epoch 6/6
Epoch = 6, Prev LR = 0.0024, New LR = 0.0022
 - 30s - loss: 0.3950 - acc: 0.8463 - val_loss: 1.1845 - val_acc: 0.5451
train-test f1 correlation = 
           f1_train   f1_test
f1_train  1.000000  0.973384
f1_test   0.973384  1.000000


,p1_by_p0,p2_by_p0,acc_train,f1_train,acc_test,f1_test
2550,1.000000,1.000000,0.722736,0.730502,0.544304,0.465280
17,0.250000,2.827338,0.902439,0.858828,0.723892,0.524632
12,0.250000,2.082666,0.897393,0.859721,0.702532,0.524576
16,0.250000,2.659660,0.903000,0.861232,0.722310,0.524155
63,0.269547,2.082666,0.896552,0.859416,0.699367,0.523918


Accuracy = 0.74, Macro F1 = 0.75
Accuracy = 0.53, Macro F1 = 0.44
train-test f1 correlation = 
           f1_train   f1_test
f1_train  1.000000  0.895394
f1_test   0.895394  1.000000


,p1_by_p0,p2_by_p0,acc_train,f1_train,acc_test,f1_test
2550,1.000000,1.000000,0.742896,0.752897,0.530303,0.442341
4,0.250000,1.277046,0.887957,0.866091,0.667929,0.494212
55,0.269547,1.277046,0.885250,0.863768,0.664141,0.494145
157,0.313347,1.277046,0.869824,0.850049,0.654040,0.492254
3,0.250000,1.201309,0.884709,0.862717,0.662247,0.491990


              precision    recall  f1-score   support

           0       0.30      0.28      0.29       174
           1       0.27      0.77      0.40       239
           2       0.83      0.52      0.64      1171

   micro avg       0.53      0.53      0.53      1584
   macro avg       0.47      0.52      0.44      1584
weighted avg       0.68      0.53      0.56      1584



```
Accuracy = 0.89, Macro F1 = 0.86
Accuracy = 0.67, Macro F1 = 0.55
              precision    recall  f1-score   support

           0       0.34      0.47      0.39       184
           1       0.42      0.55      0.47       249
           2       0.84      0.73      0.78      1151

   micro avg       0.67      0.67      0.67      1584
   macro avg       0.53      0.58      0.55      1584
weighted avg       0.71      0.67      0.69      1584

```


```
Accuracy = 0.92, Macro F1 = 0.89
Accuracy = 0.67, Macro F1 = 0.53
              precision    recall  f1-score   support

           0       0.31      0.37      0.34       199
           1       0.44      0.52      0.48       248
           2       0.81      0.76      0.78      1137

   micro avg       0.67      0.67      0.67      1584
   macro avg       0.52      0.55      0.53      1584
weighted avg       0.69      0.67      0.68      1584
```

```
Accuracy = 0.92, Macro F1 = 0.90
Accuracy = 0.66, Macro F1 = 0.39
Accuracy = 0.67, Macro F1 = 0.51
```

In [167]:
reset_keras()
gc.collect()
model = build_model()
metrics = train_generator_model_with_validation(model,df,None,extract_values,[1,1,1],
                   **conv_embedded_params["training_policy"])


Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7f97a34c6ba8>>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1455, in __del__
    self._session._session, self._handle, status)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/errors_impl.py", line 528, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.errors_impl.CancelledError: Session has been closed.


901071


0

FC Input dims =  (None, 552)
Before Resampling, Size =  (5279, 22) 
 2    3825
1     837
0     617
Name: sentiment, dtype: int64
After Resampling =  (13783, 22) 
 1    5022
0    4936
2    3825
Name: sentiment, dtype: int64
Total Model Params =  1092551
Epoch 1/6
Epoch = 1, Prev LR = 0.0030, New LR = 0.0002
216/216 [==============================] - 58s 267ms/step - loss: 1.2362 - acc: 0.3508


,p1_by_p0,p2_by_p0,f1_train
887,1.0,1.00,0.192608
851,1.0,1.00,0.192608
78,0.9,2.00,0.419780
77,0.9,1.75,0.414661
856,1.0,2.25,0.408831
855,1.0,2.00,0.408312


...

,p1_by_p0,p2_by_p0,f1_train
851,1.0,1.00,0.485728
887,1.0,1.00,0.485728
67,0.7,8.50,0.755897
66,0.7,8.25,0.754202
68,0.7,8.75,0.753862
26,0.5,7.50,0.752886


Epoch 5/6
Epoch = 5, Prev LR = 0.0027, New LR = 0.0024
216/216 [==============================] - 42s 197ms/step - loss: 0.4676 - acc: 0.8130


,p1_by_p0,p2_by_p0,f1_train
851,1.0,1.00,0.635535
887,1.0,1.00,0.635535
16,0.5,5.00,0.853032
20,0.5,6.00,0.852849
19,0.5,5.75,0.852370
15,0.5,4.75,0.851841


Epoch 6/6
Epoch = 6, Prev LR = 0.0024, New LR = 0.0022
216/216 [==============================] - 43s 197ms/step - loss: 0.3988 - acc: 0.8466


,p1_by_p0,p2_by_p0,f1_train
851,1.0,1.00,0.764920
887,1.0,1.00,0.764920
85,0.9,3.75,0.868378
863,1.0,4.00,0.867891
862,1.0,3.75,0.867645
864,1.0,4.25,0.867609


In [168]:
y_test_preds = model.predict(extract_values(df_test))

Test Predictions - No cutout


In [173]:


y_test_preds_out = prediction_caliberation_wrapper(y_test_preds,[9,2,1])
y_test_preds_out = np.argmax(y_test_preds_out, axis=1)
assert len(y_test_preds_out)==df_test.shape[0]
df_test['sentiment'] = y_test_preds_out
df_test[['unique_hash','sentiment']].to_csv("submission.csv",index=False)





# M2: CNN Model - No Padding and Embedding layer

In [302]:
df_np = read_csv("train_non_padded.csv")
df_test_np = read_csv("test_non_padded.csv")

df_np.shape
df_test_np.shape

(5279, 16)

(2924, 14)

In [356]:
reset_keras()
trained_embeddings = PreTrainedEmbeddingsTransformer(fb_model,size=fasttext_dims)

410


In [357]:


def extract_values_pretrained(x):
    x1 = x["full_txt"].values
    x2 = x["full_txt_mask"].values
    x3 = x["full_txt_mask_gaussian"].values
    x4 = x["context_txt"].values
    x5 = x[["occurences_count","review_length","review_length_by_occurences_count"]].values
    train_phase = False
    if 'ohe_labels' in x.columns:
        train_phase = True
        y = np.stack(x['ohe_labels'].values,axis=0)

    

    full_txt_maxlen = 0
    for x in x1:
        full_txt_maxlen = max(full_txt_maxlen,len(x))
        
    context_txt_maxlen = 0
    for x in x4:
        context_txt_maxlen = max(context_txt_maxlen,len(x))
    
    x1 = list(pad_text_sequences(x1, maxlen=full_txt_maxlen,jobs=1))
    x2 = list(pad_text_sequences(x2, maxlen=full_txt_maxlen,empty=0.0,jobs=1))
    x3 = list(pad_text_sequences(x3, maxlen=full_txt_maxlen,empty=0.0,jobs=1))
    x4 = list(pad_text_sequences(x4, maxlen=context_txt_maxlen,jobs=1))
    
    x1 = np.array(x1)
    x4 = np.array(x4)
    
    x1 = np.stack(x1, axis=0)
    x4 = np.stack(x4, axis=0)
    
    x2 = np.asarray([np.asarray(x) for x in x2])
    x3 = np.asarray([np.asarray(x) for x in x3])
    
#     print(x1.shape,x2.shape,x3.shape,x4.shape)
    if train_phase:
        x1,x2,x3 = batch_cutout( x1,x2,x3,p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)
        
        x4 = batch_cutout(x4, p=cutout_proba, min_words=min_cutout, max_words=max_cutout,)[0]
    
    
    x4 = trained_embeddings.transform(x4)
    x1 = trained_embeddings.transform(x1)
    
    # Batch padding!!
    
    
    
    
    
    
    
#     x2 = np.asarray([np.asarray(x).reshape((len(x),1)) for x in x2])
#     x3 = np.asarray([np.asarray(x).reshape((len(x),1)) for x in x3])
#     x2 = np.stack(x2, axis=0)
#     x3 = np.stack(x3, axis=0)
    x2 = x2.reshape((x2.shape[0],x2.shape[1],1))
    x3 = x3.reshape((x3.shape[0],x3.shape[1],1))
#     print(x1.shape,x2.shape,x3.shape,x4.shape)
    if train_phase:
        return [x1,x2,x3,x4,x5],y
    else:
        return [x1,x2,x3,x4,x5]



In [317]:
sample = df_np.sample(1000)
_ = extract_values_pretrained(sample)

gc.collect()

(1000, 7204) (1000, 7204) (1000, 7204) (1000, 1267)
(1000, 7204, 300) (1000, 7204, 1) (1000, 7204, 1) (1000, 1267, 300)


1656

In [358]:
def build_model_no_embedding():
    full_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    full_txt_mask = Input(shape=(None,1), dtype='float32')
    full_txt_gaussian = Input(shape=(None,1), dtype='float32')
    
    context_txt_input = Input(shape=(None,fasttext_dims), dtype='float32')
    
    numeric_inputs = Input(shape=(3,),name="numeric_input", dtype='float32')
    
    fc_layer_width = conv_non_embedded_params["fc_layer_width"]
    full_text_conv_layer_width = conv_non_embedded_params["full_text_conv_layer_width"]
    context_conv_layer_width = conv_non_embedded_params["context_conv_layer_width"]
    fc_layer_depth = conv_non_embedded_params["fc_layer_depth"]
    
    
    full_txt_embedding_out = full_txt_input
    context_txt_embedding_out = context_txt_input # 25 channels
    
    full_txt_embedding_out = Dropout(0.1)(full_txt_embedding_out)
    context_txt_embedding_out = Dropout(0.1)(context_txt_embedding_out)
    # print(K.int_shape(full_txt_embedding_out))
    
    # https://stackoverflow.com/questions/53849829/element-wise-multiplication-with-keras
    gaussian_mask_out = Lambda(lambda x: x[0] * x[1])([full_txt_embedding_out, full_txt_gaussian])
    # print(K.int_shape(gaussian_mask_out))
    full_txt_gaussian_out = full_txt_gaussian
    full_txt_mask_out = full_txt_mask
    # print(K.int_shape(full_txt_mask_out))
    
    fc_input1 = GlobalAveragePooling1D()(gaussian_mask_out)
    fc_input1 = fc_layer(fc_input1,fc_layer_width,)
    
    full_txt_embedding_masked = concatenate([full_txt_mask_out,full_txt_embedding_out,full_txt_gaussian_out]) # 27 channels now
    # print(K.int_shape(full_txt_embedding_masked))
    
    fc_input2 = conv_pipe(context_txt_embedding_out, context_conv_layer_width)
    fc_input3 = conv_pipe(full_txt_embedding_masked, full_text_conv_layer_width)
    
    
    numerics = fc_layer(numeric_inputs,8,)
    numerics = fc_layer(numerics,8,)
    
    fc_input = concatenate([numerics,fc_input1,fc_input2,fc_input3])
    fc_input = Dropout(0.1)(fc_input)
    
    for i in range(fc_layer_depth-1):
        fc_input = fc_layer(fc_input,fc_layer_width)
    
    fc_out = fc_layer(fc_input,int(fc_layer_width/2),bn=False)
    fc_out = Dense(3)(fc_out)
    out = Activation("softmax")(fc_out)
    
    model = Model(inputs=[full_txt_input,full_txt_mask,full_txt_gaussian,context_txt_input,numeric_inputs], 
                  outputs=[out])
    
    return model


In [359]:
gc.collect()
print(conv_non_embedded_params)
looped_random_validation(df_np,build_model_no_embedding,train_generator_model_with_validation,extract_values_pretrained,
                        **conv_non_embedded_params["training_policy"])


0

{'full_text_conv_layer_width': 6, 'context_conv_layer_width': 4, 'fc_layer_width': 80, 'fc_layer_depth': 2, 'training_policy': {'epochs': 10, 'batch_size': 32, 'policy': 'adam', 'lr': 0.005, 'max_lr_olr': 0.002, 'lr_shed_fn': <function <lambda> at 0x7fb5e24157b8>}}
Before Resampling, Size =  (3695, 16) 
 2    2696
1     578
0     421
Name: sentiment, dtype: int64
After Resampling =  (8533, 16) 
 0    2947
1    2890
2    2696
Name: sentiment, dtype: int64
Epoch 1/10
 70/267 [======>.......................] - ETA: 5:22 - loss: 1.2108 - acc: 0.3379

ResourceExhaustedError: OOM when allocating tensor with shape[32,13735,96] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/Adam/gradients/zeros_73}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
model = build_model_no_embedding()
train_generator_model_with_validation(model,df_np,None,extract_values_pretrained,
                   **conv_non_embedded_params["training_policy"])


In [ ]:
iterator = MakeIter(df_test_np,extract_values,batch_size)
y_test_preds = model.predict_generator(iterator,steps=len(iterator))
y_test_preds = np.argmax(y_test_preds, axis=1)
df_test_np['sentiment'] = y_test_preds
df_test_np[['unique_hash','sentiment']].to_csv("submission.csv",index=False)
